**Imports**

In [1]:
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
import os
import numpy
from sklearn.preprocessing import LabelEncoder

**Reading files**

In [2]:
X_fileName = []
y_val = []

In [3]:
for fileName in sorted(glob.iglob('near-infrared/**/train_pose/**/**', recursive=False)):
    X_fileName.append(fileName)
    y_val.append(fileName.split("\\")[-2])

In [4]:
myDf = pd.DataFrame()
myDf['fileName'] = X_fileName
myDf['label'] = y_val

In [5]:
myLabelEncoder = LabelEncoder()
myDf['label_val'] = myLabelEncoder.fit_transform(myDf['label'])

**PreProcessing**

In [6]:
def pointremove(image):
    
    img=image.copy()
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    for i in range(1, nb_components):
        if sizes[i]<100 :
            img[output==i]=0
    return img

In [7]:
def maxmeanpreprop(path):
    img1 = cv2.imread(path)
    g_img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img = g_img.copy()
    ret1,th1 = cv2.threshold(g_img,40,255,cv2.THRESH_BINARY)
    th1=pointremove(th1)
    blur = cv2.GaussianBlur(th1,(5,5),0)
    ret2,th2 = cv2.threshold(blur,130,255,cv2.THRESH_OTSU)

    a_img=th2&img
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(a_img, connectivity=4)
    sizes = stats[:, -1]

    max_label = 0
    max_mean = 0

    for i in range(1, nb_components):
        mean=np.mean(a_img[output==i])
        if mean > max_mean and sizes[i]>1000 :
            max_label = i
            max_mean = mean

    img3 = np.zeros(output.shape)
    img3[output == max_label] = 255
    new_image = img3[stats[max_label][1]:stats[max_label][1]+stats[max_label][3],stats[max_label][0]:stats[max_label][0]+stats[max_label][2]]
    image_for_input = cv2.resize(new_image, (100, 100), interpolation = cv2.INTER_NEAREST)
    return image_for_input

**Creating my generator**

In [8]:
def my_generator(features, batch_size=128):
    # Create empty arrays to contain batch of features and labels#
    batch_features = np.zeros((batch_size, 100, 100, 1))
    batch_labels = np.zeros((batch_size))
    while True:
        for i in range(batch_size):
            # choose random index in features
            index = np.random.choice(len(features),1)
            batch_features[i] = maxmeanpreprop(features.loc[index[0]]['fileName']).reshape([100, 100, 1])
            batch_labels[i] = features.loc[index[0]]['label_val']
        yield batch_features, batch_labels

**Model** 

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K

Using TensorFlow backend.
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\akkir\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [10]:

def MyModel(input_shape):
    
    model = Sequential()

    model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=input_shape))

    model.add(Conv2D(24, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(36, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(48, 5, 5, activation='relu', subsample=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(Dropout(0.1))

    model.add(Flatten())

    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(10, activation='relu'))
    model.add(Dense(16, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  

    return model

In [11]:
myMod = MyModel([100, 100, 1])

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  del sys.path[0]
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  app.launch_new_instance()
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


In [12]:
os.makedirs('./model', exist_ok=True)
for i in range(4):
    myMod.fit_generator(my_generator(myDf), samples_per_epoch=100, nb_epoch=50)
    myMod.save('./model/keras_model'+str((i+1)*50)+'.h5')

C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\akkir\anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=100, epochs=50)`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/50
100/100 [==============================] - 173s 2s/step - loss: 2.6830 - accuracy: 0.1291
Epoch 2/50
100/100 [==============================] - 159s 2s/step - loss: 2.2638 - accuracy: 0.2747
Epoch 3/50
100/100 [==============================] - 159s 2s/step - loss: 1.8280 - accuracy: 0.3923
Epoch 4/50
100/100 [==============================] - 158s 2s/step - loss: 1.5346 - accuracy: 0.4953
Epoch 5/50
100/100 [==============================] - 157s 2s/step - loss: 1.3161 - accuracy: 0.5782
Epoch 6/50
100/100 [==============================] - 157s 2s/step - loss: 1.1383 - accuracy: 0.6383
Epoch 7/50
100/100 [==============================] - 182s 2s/step - loss: 1.0157 - accuracy: 0.6803
Epoch 8/50
100/100 [==============================] - 205s 2s/step - loss: 0.8711 - accuracy: 0.7266
Epoch 9/50
100/100 [==============================] - 201s 2s/step - loss: 0.8024 - accuracy: 0.7602
Epoch 10/50
100/100 [==============================] - 199s 2s/step - loss: 0.7225 - accura

100/100 [==============================] - 150s 1s/step - loss: 0.1785 - accuracy: 0.9473
Epoch 32/50
100/100 [==============================] - 148s 1s/step - loss: 0.1728 - accuracy: 0.9492
Epoch 33/50
100/100 [==============================] - 149s 1s/step - loss: 0.1926 - accuracy: 0.9455
Epoch 34/50
100/100 [==============================] - 174s 2s/step - loss: 0.1850 - accuracy: 0.9483
Epoch 35/50
100/100 [==============================] - 194s 2s/step - loss: 0.1772 - accuracy: 0.9478
Epoch 36/50
100/100 [==============================] - 189s 2s/step - loss: 0.1896 - accuracy: 0.9431
Epoch 37/50
100/100 [==============================] - 199s 2s/step - loss: 0.1813 - accuracy: 0.9465
Epoch 38/50
100/100 [==============================] - 190s 2s/step - loss: 0.1821 - accuracy: 0.9462
Epoch 39/50
100/100 [==============================] - 189s 2s/step - loss: 0.1653 - accuracy: 0.9515
Epoch 40/50
100/100 [==============================] - 191s 2s/step - loss: 0.1727 - accuracy:

100/100 [==============================] - 152s 2s/step - loss: 0.1292 - accuracy: 0.9606
Epoch 12/50
100/100 [==============================] - 155s 2s/step - loss: 0.1248 - accuracy: 0.9609
Epoch 13/50
100/100 [==============================] - 168s 2s/step - loss: 0.1270 - accuracy: 0.9609
Epoch 14/50
100/100 [==============================] - 174s 2s/step - loss: 0.1228 - accuracy: 0.9623
Epoch 15/50
100/100 [==============================] - 176s 2s/step - loss: 0.1305 - accuracy: 0.9612
Epoch 16/50
100/100 [==============================] - 188s 2s/step - loss: 0.1196 - accuracy: 0.9654
Epoch 17/50
100/100 [==============================] - 188s 2s/step - loss: 0.1277 - accuracy: 0.9621
Epoch 18/50
100/100 [==============================] - 184s 2s/step - loss: 0.1218 - accuracy: 0.9618
Epoch 19/50
100/100 [==============================] - 187s 2s/step - loss: 0.1227 - accuracy: 0.9638
Epoch 20/50
100/100 [==============================] - 156s 2s/step - loss: 0.1202 - accuracy:

In [13]:
from keras import backend as K
K.set_learning_phase(0)

In [14]:
from keras.models import load_model
model = load_model('./model/keras_model200.h5')

In [15]:
from keras import backend as K
import tensorflow.compat.v1 as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 158 variables.
INFO:tensorflow:Converted 158 variables to const ops.


In [16]:
tf.train.write_graph(frozen_graph, "model", "tf_model200.pb", as_text=False)

'model\\tf_model200.pb'

In [17]:
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    with gfile.FastGFile("./model/tf_model200.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in=tf.import_graph_def(graph_def)
        

Instructions for updating:
Use tf.gfile.GFile.


In [18]:
X_file=[]
Y_va=[]
for fileName in sorted(glob.iglob('near-infrared/**/test_pose/**/**', recursive=False)):
    X_file.append(fileName)
    Y_va.append(fileName.split("\\")[-2])

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def testing(input_size):
    X=np.zeros((input_size,100,100,1))
    Y=np.zeros((input_size))
    
    Y1=myLabelEncoder.transform(Y_va)    
    k=-1
    for i in range(len(X_file)):
        k=k+1
        if (k==input_size):
            break
        image=maxmeanpreprop(X_file[i]).reshape(100,100,1)
        X[i]=image
        Y[i]=Y1[i]
        
    prediction = model.predict(X)
    compare_x=[]
    
    for i in range(input_size):
        result=np.where(prediction[i] == max(prediction[i]))
        if result[0] !=None:
            compare_x.append(result[0][0])
        else :
            compare_x.append(-1)        
    accuracy = accuracy_score(Y,compare_x)
    print('Accuracy: %f' % accuracy)
    #f1 = f1_score(Y,compare_x,pos_label='positive',average='micro')
    #print('F1 score: %f' % f1)
    print("Confusion Matrix :")
    print(confusion_matrix(compare_x,Y))
    return


In [22]:
testing(22579)

Accuracy: 0.946853
Confusion Matrix :
[[1353    5   16    1    1    0    1    0    0   11   12    8    0    0
     0    0]
 [   0 1352    0    0    2    0    6    1   24    0    1    0    0    1
     5    0]
 [   0    0  862    6   22    0   14    0    1    0   35    4    0    0
     0    0]
 [   7    2   60 1282    3    0    2    1    0    8   12    9    2    1
     1    4]
 [   0    4    0    0 1186   24    0    0    0    1   16    4    0    1
     0    0]
 [   1    1    0    0    7 1187    0    0    0    2    0    1    8   18
     1    0]
 [   0    1    9    2    5    3 1220   16    0    3    6    1    0    0
     2    1]
 [   2    0    0    8    0    0    8 1324    1    5    0    2    0    1
     0    0]
 [   1   12    0    0    0    0    3    5 1285    5    1   10    0    0
     5    0]
 [   1    9    5    2    2    1    4    0    0 1342    1    3    0    0
     2    0]
 [   2    4   21    0   12    0    0    0    0    0 1937   92    3    0
     0    0]
 [   2    0    7   28    3 